In [81]:
import pickle
import nltk

In [82]:
from tqdm import tqdm

MAX_DISH_COUNT = 100

with open('./categories/Chinese.pkl', 'rb') as f:
    cuisine_reviews = pickle.load(f)    
with open('ChineseDishes.txt', 'r') as f:
    dishes = {line.replace("\n", "").lower() : {} for line in f.readlines()[0:MAX_DISH_COUNT]}

for _review in tqdm(cuisine_reviews):
    _review['sentences'] = nltk.sent_tokenize(_review['text'])

100%|██████████| 38716/38716 [00:10<00:00, 3846.92it/s]


In [83]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

for _dish in tqdm(dishes):
    dishes[_dish] = {'rest2review':{}, 'review2polarity':{}, 'rest2polarity':{}}
    _dish2rest = set()
    _review_count = 0
    
    for _review in cuisine_reviews:
        _review_id = _review['review_id']
        _rest_id = _review['business_id']
        _review_polarity = 0
        _review_sentence_count = 0
        for _sentence in _review['sentences']:
            _count = _sentence.count(_dish)
            if _count > 0:
                _review_sentence_count += 1
                _ss = sia.polarity_scores(_sentence)
                _review_polarity += _ss['compound']
        
        if _review_sentence_count > 0:
            _review_polarity = _review_polarity / _review_sentence_count
            _review_count += 1
            dishes[_dish]['review2polarity'][_review_id] = _review_polarity
            if _rest_id in dishes[_dish]['rest2review']:
                dishes[_dish]['rest2review'][_rest_id].add(_review_id)
                dishes[_dish]['rest2polarity'][_rest_id] += _review_polarity
            else:
                dishes[_dish]['rest2review'][_rest_id] = set([_review_id])
                dishes[_dish]['rest2polarity'][_rest_id] = _review_polarity
    
            
    for _id in dishes[_dish]['rest2polarity']:
        dishes[_dish]['rest2polarity'][_id] /= len(dishes[_dish]['rest2review'][_id])

100%|██████████| 100/100 [00:27<00:00,  3.65it/s]


In [134]:
popular_dishes = {dish:{'review_count':len(dishes[dish]['review2polarity']), 
                        'rest_count':len(dishes[dish]['rest2review']),
                       'polarity':sum([dishes[dish]['review2polarity'][i] for i in dishes[dish]['review2polarity']])/len(dishes[dish]['review2polarity'])} 
                  for dish in dishes}

def by_rest_count(dish):
    return popular_dishes[dish]['rest_count']

def by_review_count(dish):
    return popular_dishes[dish]['review_count']
    
sorted_dishes = sorted(popular_dishes, key=by_rest_count, reverse=True)

In [136]:
dishes['chow mien']['rest2polarity']

{'-3WVw1TNQbPBzaKCaQQ1AQ': 0.3612,
 '-Gse8l_ec33gkWgotCLsew': 0.0,
 '-mz0Zr0Dw6ZASg7_ah1R8A': 0.17976666666666663,
 '0LBv0QQK9L6ArPkO4eXjJA': 0.23570000000000002,
 '0nyaKW_6QHSJ74OVnDmyhA': 0.0,
 '1ldDbggHHgPlj3k0SpBTKQ': 0.6071,
 '23SJEjfyU5gxY2chAKhgcw': 0.0,
 '2Dbg35sPSGmZGcbBVNa9dA': -0.5423,
 '2sn5BECdlYxqQlw1VODDoA': 0.2996333333333333,
 '3RKhnCtiN6jfaSbJU-9kmA': 0.636,
 '5OX1kKlNHCaDmGq4qrFBmQ': 0.765,
 '5W48_DnrXVD7EbtmE4pxOQ': 0.5719,
 '5YgXxBUzhf3qh8F8wYSqsA': 0.8555,
 '5kOYuJtRjAUWZGDkP8vnRw': 0.0,
 '7opoNCBk1B00claYpFx4ug': 0.8555,
 '8SkoyQ3iHXECEit2BDwBrw': -0.1366,
 '8TGUK7LD3M-vLWml-jLb0w': 0.644,
 '8z2DkKf_FrPVNtt_5w4QsQ': 0.8717,
 'A4pL6j3e6JAtblmCUF2GZA': 0.0,
 'AumuGuTg3HqvtwbLPi-ZCg': 0.4215,
 'AxScnvjfLqpPmTLXGjNhcA': 0.0,
 'BicQSnGZnY_yuGEDkhVfnA': 0.0,
 'BnVJmCTNrO8ZtbC1vkv9Zg': -0.12703333333333333,
 'C4CbD-0Rz4nglavf_0dRdg': -0.6808,
 'Co-sJv4GNxILkF79ZMLq-Q': 0.0,
 'CqKiF2X0_4erH02-CUiweQ': -0.0386,
 'CwTYM1-zs80BB1AWJrWmxQ': -0.34,
 'DL3coobcfFjXV_F2zV7sLw': 